 # 웹크롤링과 텍스트데이터 시각화

### Outline

### 5. Get 방식을 이용한 웹크롤링
    5.1 특정한 URL의 정보 추출
    5.2 연결된 URL의 정보 추출

#### 5.1 특정한 URL의 정보 추출

- 웹크롤링을 하려고 하는 웹사이트의 URL 지정

In [ ]:
url = 'https://search.naver.com/search.naver?where=blog&sm=tab_viw.blog&query=연세대학교&nso='

In [ ]:
from IPython.display import Image
Image(filename='Yonsei.png', width=700)

- Get 방식을 이용해 서버에 정보 요청
- Requests 라이브러리 설치

In [ ]:
import requests as req # 웹크롤링을 위한 파이썬 라이브러리

In [ ]:
res = req.get(url = url) # Get 방식을 이용한 정보 요청

- 응답받은 HTML 코드의 내용을 확인

In [ ]:
print(res.text)

- HTML 코드를 파싱
- BeautifulSoup 라이브러리 설치

In [ ]:
!pip install --upgrade pip # pip는 (아나콘다에 포함되지 않은) 외부 라이브러리를 설치하는 명령어
!pip install bs4           # 주피터노트북에서 !(느낌표)를 앞에 붙이면 커맨드 창에서 실행하는 것과 동일함

In [ ]:
from bs4 import BeautifulSoup as bs # html을 쉽게 다룰 수 있는 파이썬 라이브러리

In [ ]:
soup = bs(res.text,'html.parser') # html을 파싱

- 파싱한 HTML 코드에서 원하는 정보 추출

In [ ]:
html = soup.select('div > a.api_txt_lines.total_tit') # 원하는 부분의 CSS 선택자 지정

In [ ]:
html[2] # 리스트의 3번째 요소 내용 확인

In [ ]:
html[2].text # 블로그 제목 추출

In [ ]:
html[2].get('href') # 하이퍼링크를 걸어준 HTML 문서

- 각 블로그의 제목 가져오기

In [ ]:
title = [x.text for x in html]
print(title)

In [ ]:
out = []
for x in html:
    out += [x.text]
urls = out
print(urls)

- 각 블로그의 주소 가져오기

In [ ]:
urls = [x.get('href') for x in html]
print(urls)

#### 5.2 연결된 URL의 정보 추출

- 특정한 URL과 하이퍼링크로 연결된 URL의 정보 추출
- 블로그 검색 후, 각 블로그의 내용을 추출
- 우리가 일반적으로 방문하는 웹사이트에서는 웹크롤링이 불가능한 경우 발생

In [ ]:
url = html[2].get('href')
url

- 웹사이트의 동일한 정보를 가지고 있는 링크 추출
- 블로그의 아이디와 글 번호는 동일

In [ ]:
'https://blog.naver.com/PostView.nhn?blogId=yonseiblog&logNo=222190336768&redirect=Dlog&widgetTypeCall=true&topReferer=https%3A%2F%2Fsearch.naver.com%2Fsearch.naver%3Fwhere%3Dblog%26sm%3Dtab_viw.blog%26query%3D%25EC%2597%25B0%25EC%2584%25B8%25EB%258C%2580%25ED%2595%2599%25EA%25B5%2590%26nso%3D&directAccess=false'

- 이전에 추출한 블로그의 주소에서 블로그의 아이디와 글 번호만 추출

In [ ]:
import re

In [ ]:
def id_extract(url):
    blog_name = re.sub('.+com/|\?.+','',url)
    article_id = re.sub('.+logNo=','',url)
    return blog_name,article_id

- 이전에 추출한 블로그의 주소를 넣으면 자동으로 내용을 추출하는 함수 생성
- **주의**: 반복적으로 내용을 추출하는 경우, 파이썬의 속도가 지나치게 빠름

In [ ]:
import time # 시간과 관련된 함수 모듈

def blog_crawler(url):
    try:
        blog_name,article_id = id_extract(url)
        new_url = 'https://blog.naver.com/PostView.nhn?blogId='+blog_name+'&logNo='+article_id+'&redirect=Dlog&widgetTypeCall=true&topReferer=https%3A%2F%2Fsearch.naver.com%2Fsearch.naver%3Fwhere%3Dblog%26sm%3Dtab_viw.blog%26query%3D%25EC%2597%25B0%25EC%2584%25B8%25EB%258C%2580%25ED%2595%2599%25EA%25B5%2590%26nso%3D&directAccess=false'
        res  = req.get(new_url)
        soup = bs(res.text,'html.parser')
        time.sleep(1) # 파이썬 일시정지 함수
        return [x.text for x in soup.select('div.se-main-container')]
    except:
        return ""

In [ ]:
id_extract(url)

In [ ]:
blog_crawler(url)

- 여러 개의 블로그에서 한번에 내용을 추출

In [ ]:
# 형식에 맞는 블로그 주소만 추출
urls = [x for x in urls if x[:23] == 'https://blog.naver.com/']

In [ ]:
blog = [blog_crawler(url) for url in urls]
blog = [x for x in blog if len(x)>0]

In [ ]:
blog